In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense
from tensorflow.keras.preprocessing import sequence
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  # Pour afficher les graphs dans le notebook

In [ ]:
max_words = 5000  # Vocabulaire limité aux 5 000 mots les plus fréquents
max_len = 200     # Longueur maximale des critiques

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_words)
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

In [ ]:
def build_model(rnn_type):
    model = Sequential()
    model.add(Embedding(max_words, 128))
    
    if rnn_type == 'simple':
        model.add(SimpleRNN(64))
    elif rnn_type == 'lstm':
        model.add(LSTM(64))
    else:
        model.add(GRU(64))
        
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
    return model

In [ ]:
plt.figure(figsize=(10, 5))

for rnn_type in ['simple', 'lstm', 'gru']:
    model = build_model(rnn_type)
    print(f"\n=== Entraînement {rnn_type.upper()} ===")
    history = model.fit(X_train, y_train,
                       batch_size=128,
                       epochs=5,
                       validation_data=(X_test, y_test),
                       verbose=1)
    
    # Sauvegarde du modèle
    model.save(f'{rnn_type}_model.keras')
    
    # Courbe de performance
    plt.plot(history.history['val_accuracy'], label=rnn_type.upper())

plt.title('Comparaison des performances')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
word_index = imdb.get_word_index()
reverse_word_index = {v: k for k, v in word_index.items()}

def predict_sentiment(model, text):
    words = text.lower().split()
    encoded = [word_index[word] for word in words if word in word_index]
    padded = sequence.pad_sequences([encoded], maxlen=max_len)
    pred = model.predict(padded)[0][0]
    return "POSITIF" if pred > 0.5 else "NEGATIF", pred

test_phrases = [
    "This movie was a masterpiece of cinema",
    "I've never seen something so terrible",
    "The film had good ideas but poor execution"
]

for rnn_type in ['simple', 'lstm', 'gru']:
    model = tf.keras.models.load_model(f'{rnn_type}_model.keras')
    print(f"\n=== Prédictions {rnn_type.upper()} ===")
    for phrase in test_phrases:
        sentiment, score = predict_sentiment(model, phrase)
        print(f"{phrase[:50]}... → {sentiment} ({score:.2f})")